Скрипт который рисует HTML и PNG график любого тайла. Для сравнения взяли два 1101 и 2203

In [53]:
from Bio import SeqIO
import plotly.express as px
import gzip

input_file = "C:\Work_place\Projects\Bubble_Off\Fastq_files\\amp_res_1.fastq.gz"
tile_name = ['1101', '2203'] # Name of Tiles
limit = ['100', 'all'] # Amount of reads

# Open the gzipped input file and process it as a FastQ file
file = gzip.open(input_file, 'rt')
file = SeqIO.parse(file, 'fastq')
status = []

for tile in tile_name:
    for lim in limit:

        list_tile = []
        list_x_pos = []
        list_y_pos = []
        list_number_quality = []
        list_color_quality = []
        list_z_pos = []
        j = 0

        for record in file:
                first_line, read, _, quality, _ = record.format('fastq').split('\n')
                srr_number, sequence_id, length = first_line.split(' ')
                instrument, run_number, flowcell_id, number_lane, number_tile, x_pos, y_pos, *other = sequence_id.split(':')

                if number_tile == tile:
                    for i, number_quality in enumerate(record.letter_annotations["phred_quality"]):
                        j+=1
                        list_x_pos.append(int(x_pos))
                        list_y_pos.append(int(y_pos))
                        list_z_pos.append(i+1)
                        list_number_quality.append(number_quality)

                        if number_quality >= 28:
                            list_color_quality.append('g')
                        elif number_quality >= 20:
                            list_color_quality.append('y')
                        else:
                            list_color_quality.append('r')

                if number_tile > tile:
                    break

                if lim != 'all':
                    if j > int(lim)*100:
                        break


        x_values = list_x_pos
        y_values = list_y_pos
        z_values = list_z_pos
        size_markers = [10] * len(list_z_pos)


        fig = px.scatter_3d(x=x_values, y=y_values, z=z_values,
                            color = list_number_quality,
                            labels={'x':'X','y':'Y', 'z':'Length, nt.', 'color': 'Phred score'},
                            color_continuous_scale= px.colors.diverging.RdYlGn,
                            title='Tile_' + tile,
                            size= size_markers, size_max=7, width=1000, height=1000)

        fig.update_traces(marker=dict(line=dict(width=0)))

        fig.write_html(f'..\Graphs\Tile_%s(%s).html'%(tile,lim))
        fig.write_image(f'..\Graphs\Tile_%s(%s).png'%(tile,lim))

        # fig.show()
        print(f'Tile %s with %s reads is done'%(tile, lim))

Tile 1101 with 100 reads is done
Tile 1101 with all reads is done
Tile 2203 with 100 reads is done
Tile 2203 with all reads is done
